In [1]:
# ================================================== [ setting ] =======================================================
import pandas as pd 
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold 
from xgboost import XGBClassifier
import lightgbm as lgb 
from bayes_opt import BayesianOptimization
from functools import partial 

import os 
os.chdir("C:/Users/knuser/Desktop/Santander Customer Satisfaction")

C:\Users\knuser\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
# =============================================== [ preprocessing ] ====================================================
def Preprocessing(file_path) : 
    df = pd.read_csv(file_path)
    df["var3"] = df["var3"].replace(-999999, np.median(df["var3"]))
    df.drop("ID", axis = 1, inplace = True)
    
    X_features = df.iloc[:, :-1]
    y_labels = df.iloc[:, -1]
    
    return df, X_features, y_labels  

train_df, X_features, y_labels = Preprocessing("train.csv")
test_df, _, _ = Preprocessing("test.csv")

In [22]:
# ================================================= [ modeling ] =======================================================
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data = None, y_data = None, n_splits = 5, output = "socre") :
    score = 0
    kf = StratifiedKFold(n_splits = n_splits)
    models = []
    
    for train_index, valid_index in kf.split(x_data, y_data) : 
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = lgb.LGBMClassifier(num_leaves = int(num_leaves), 
                                   learning_rate = learning_rate, 
                                   n_estimators = int(n_estimators), 
                                   subsample = np.clip(subsample, 0, 1), 
                                   colsample_bytree = np.clip(colsample_bytree, 0, 1), 
                                   reg_alpha = reg_alpha, 
                                   reg_lambda = reg_lambda,)
        
        model.fit(x_train, y_train)
        models.append(model)
    
        pred = model.predict_proba(x_valid)[:, 1]    
        true = y_valid
        score += roc_auc_score(true, pred) / n_splits
        
    if output == "score" :
        return score
    if output == "model" :
        return models

In [23]:
func_fixed = partial(lgb_cv, x_data = X_features, y_data = y_labels, n_splits = 5, output = 'score') 

lgbBO = BayesianOptimization(
    func_fixed, 
    {'num_leaves': (16, 1024),
     'learning_rate': (0.0001, 0.1),
     'n_estimators': (16, 1024),
     'subsample': (0, 1),
     'colsample_bytree': (0, 1),
     'reg_alpha': (0, 10),
     'reg_lambda': (0, 50),}, 
    random_state = 2109
)

lgbBO.maximize(init_points = 5, n_iter = 30)

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.8187   |  0.6128   |  0.09316  |  1.001e+0 |  436.0    |  3.19     |  22.89    |  0.5532   |
|  2        |  0.8035   |  0.3464   |  0.01621  |  20.21    |  71.48    |  1.063    |  37.63    |  0.2233   |
|  3        |  0.8332   |  0.09816  |  0.02952  |  492.0    |  128.9    |  8.813    |  7.464    |  0.5725   |
|  4        |  0.8219   |  0.02332  |  0.07836  |  498.4    |  833.4    |  0.4697   |  23.6     |  0.2591   |
|  5        |  0.8124   |  0.0683   |  0.09826  |  81.96    |  432.0    |  4.108    |  35.45    |  0.5594   |
|  6        |  0.8366   |  0.7139   |  0.01559  |  495.9    |  123.3    |  2.686    |  9.645    |  0.2485   |
|  7        |  0.8389   |  0.3051   |  0.00787  |  987.6    |  25.63    |  2.371    |  47.98    |  0.01353  |
|  8      

In [35]:
params = lgbBO.max['params']
models = lgb_cv(
    params['num_leaves'], 
    params['learning_rate'], 
    params['n_estimators'], 
    params['subsample'], 
    params['colsample_bytree'], 
    params['reg_alpha'], 
    params['reg_lambda'], 
    x_data = X_features, y_data = y_labels, n_splits = 5, output = 'model') # ROC-AUC : 0.839

In [40]:
preds = []
for model in models :
    pred = model.predict_proba(test_df)[:, 1]
    preds.append(pred)
    
pred = np.mean(preds, axis = 0)

In [46]:
# ================================================= [ modeling (2) ] ===================================================
def xgb_cv(max_leaves, eta, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data = None, y_data = None, n_splits = 5, output = "socre") :
    score = 0
    kf = StratifiedKFold(n_splits = n_splits)
    models = []
    
    for train_index, valid_index in kf.split(x_data, y_data) : 
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = XGBClassifier(max_leaves = int(max_leaves),
                              eta = eta,
                              n_estimators = int(n_estimators),
                              subsample = np.clip(subsample, 0, 1),
                              colsample_bytree = np.clip(colsample_bytree, 0, 1),
                              reg_alpha = reg_alpha,
                              reg_lambda = reg_lambda, )
        
        model.fit(x_train, y_train)
        models.append(model)
    
        pred = model.predict_proba(x_valid)[:, 1]    
        true = y_valid
        score += roc_auc_score(true, pred) / n_splits
        
    if output == "score" :
        return score
    if output == "model" :
        return models

In [ ]:
xgb_func_fixed = partial(xgb_cv, x_data = X_features, y_data = y_labels, n_splits = 5, output = 'score') 

xgbBO = BayesianOptimization(
    xgb_func_fixed, 
    {'max_leaves': (16, 1024),
     'eta': (0.0001, 0.1),
     'n_estimators': (16, 1024),
     'subsample': (0, 1),
     'colsample_bytree': (0, 1),
     'reg_alpha': (0, 10),
     'reg_lambda': (0, 50),}, 
    random_state = 2109
)

xgbBO.maximize(init_points = 5, n_iter = 30)

|   iter    |  target   | colsam... |    eta    | max_le... | n_esti... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.8388   |  0.6128   |  0.09316  |  1.001e+0 |  436.0    |  3.19     |  22.89    |  0.5532   |
|  2        |  0.8298   |  0.3464   |  0.01621  |  20.21    |  71.48    |  1.063    |  37.63    |  0.2233   |
|  3        |  0.8267   |  0.09816  |  0.02952  |  492.0    |  128.9    |  8.813    |  7.464    |  0.5725   |
|  4        |  0.8222   |  0.02332  |  0.07836  |  498.4    |  833.4    |  0.4697   |  23.6     |  0.2591   |
|  5        |  0.8355   |  0.0683   |  0.09826  |  81.96    |  432.0    |  4.108    |  35.45    |  0.5594   |
|  6        |  0.8393   |  0.6629   |  0.02764  |  89.29    |  425.3    |  7.843    |  40.21    |  0.3008   |
|  7        |  0.8063   |  0.3534   |  0.02754  |  1.016e+0 |  26.33    |  9.544    |  43.1     |  0.9474   |
|  8      

In [ ]:
preds_2 = []
for model in models :
    pred = model.predict_proba(test_df)[:, 1]
    preds.append(pred)
    
preds_2 = np.mean(preds, axis = 0)

In [ ]:
total_preds = preds * 0.7 + preds_2 * 0.3 

In [ ]:
np.cortest(preds, preds_2)